In [1]:
#%%
import os
import re
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import requests
import xlwings as xw

import functions
import hide

To convert from legacy template to new template.
**Important: Item numbering is the key as some of the formula will be tied to item numbers.**

- For the "Title", it will be "Title" case
- For the "Lineitem", it will be "Title" case
- For the "Subitem", it will be "Title" case
- For the "Description", it will be sentence case

In [2]:
# Where is the file?
file = 'J12475 MODEC - 4103 PROJECT GREEN FPSO - NAV R1.xlsm'
# file = '01 Costing sheet V3.67 Rev.0.xlsm'
downloads_folder = os.path.join(os.path.expanduser('~'), 'Downloads')
file_path = Path(downloads_folder, file)

In [3]:
# Read and initialize values
# Differentiate between new and legacy template
try:
    wb = xw.Book(file_path, password=hide.legacy)
    template = 'legacy'
except:
    wb = xw.Book(file_path, password=hide.new)
    template = 'new'
visible_sheets = [sht.name for sht in wb.sheets if sht.visible]
full_column_list = ['NO', 'SN', 'Description', 'Qty', 'Unit', 'Unit Price', 'Subtotal Price', 'Scope', 'Model',
                    'Cur', 'UC', 'SC', 'Discount', 'UCD', 'SCD', 'Remark', 'Rate', 'UCDQ', 'SCDQ', 'BUCQ', 'BSCQ',
                    'Default', 'Warranty', 'Freight', 'Special', 'Risk', 'MU', 'FUP', 'RUPQ', 'RSPQ', 'UPLS', 'SPLS', 'Profit', 
                    'Margin', 'Auxiliary', 'Lumpsum', 'Flag', 'Format', 'Category', 'System']
df = pd.DataFrame(columns=full_column_list)
risk = 0.05
# Read and set currency from FX sheet
fx = wb.sheets['FX']
exchange_rates = dict(fx.range('A2:B9').value)
quoted_currency = fx.range('B12').value
project_info = dict(fx.range('A36:B46').value)
project_info = {key: value.upper() for key, value in project_info.items()}
# Read system sheets
cols = ['NO', 'Qty', 'Unit', 'Description', 'Unit Price', 'Subtotal Price', 'Model', 'Cur', 'UC', 'SC', 'Discount']
systems = pd.DataFrame()
defaults = {}
system_names = []
skip_sheets = ['FX', 'Cover', 'Intro', 'ES', 'T&C']
for sheet in visible_sheets:
    if sheet not in skip_sheets:
        system_names.append(sheet.upper())
        ws = wb.sheets[sheet]
        escalation = dict(ws.range('K2:L5').value)
        default_mu = ws.range('H5').value
        escalation['default_mu'] = default_mu
        defaults[sheet.upper()] = escalation
        last_row = ws.range('D100000').end('up').row  #Returns a number
        data = ws.range('A8:K' + str(last_row)).options(pd.DataFrame, index=False).value
        data.columns = cols
        data['System'] = str(sheet.upper())
        data['Category'] = 'Product'
        systems = pd.concat([systems, data], join='outer')
systems = pd.concat([systems, df], join='outer')
       
# Read Engineering Services
es_cols = ['NO', 'Qty', 'Unit', 'Description', 'Unit Price', 'Subtotal Price', 'Model', 'Cur', 'UC', 'SC', 'Discount']
es = wb.sheets['ES']
es_last_row = es.range('D100000').end('up').row
eng_service = es.range('A8:K' + str(es_last_row)).options(pd.DataFrame, index=False).value
eng_service.columns = es_cols
eng_service = pd.concat([eng_service, df], join='outer')
eng_service = eng_service.reindex(columns=full_column_list)
eng_service['Discount'] = np.nan
eng_service['System'] = 'ENGINEERING SERVICES'
# eng_service['Category'] = 'Service'
systems = pd.concat([systems, eng_service], join='outer')
systems = systems.reindex(columns=full_column_list)
system_names.append('ENGINEERING SERVICES')

# Set font case for some columns
systems['Unit'] = systems['Unit'].str.lower()
# systems['Scope'] = systems['Scope'].str.upper()

In [4]:
# Remove lineitem numbers
systems = systems.reset_index(drop=True)
for idx in systems.index:
    if str(systems.loc[idx, 'NO']).count('.') == 2:
        systems.loc[idx, 'NO'] = np.nan

for idx in systems.index:
    if pd.notna(systems.loc[idx, 'NO']) and not pd.notna(systems.loc[idx, 'Qty']):
        systems.loc[idx, 'NO'] = np.nan

In [5]:
# Let's take care of the main numbering
systems['Format'] = np.nan
item_count = 10
for idx in systems.index:
    if pd.notna(systems.loc[idx, 'NO']):
        try:
            systems.at[idx, 'NO'] = item_count
            systems.at[idx, 'Format'] = 'Title'
            item_count += 10
        except Exception as e:
            print(str(e))
            pass

In [6]:
# This function has been handed over to excel formula

# Let's do Serial Number
# Lineitems must have values in Description, Qty and UC.

# Reset Sub-No row as there may be some unclean data by manually numbering rows.
# systems['SN'] = np.nan
# count = 1
# for idx in systems.index: 
#     if pd.notna(systems.loc[idx, 'NO']):
#         count = 1    # Reset count

#     if pd.notna(systems.loc[idx, 'Description']) & pd.notna(systems.loc[idx, 'Qty']) & \
#         pd.notna(systems.loc[idx, 'UC']) & (systems.loc[idx, 'Format'] != 'Title'):
#         systems.at[idx, 'SN'] = count
#         systems.at[idx, 'Format'] = 'Lineitem'
#         count += 1
#     if str(systems.loc[idx, 'Description']).startswith('***'):
#         systems.at[idx, 'Format'] = 'Comment'

In [7]:
# This formula has been handed over to excel.

# Subtitle:
# for idx in systems.index:
#     if idx == 0:
#         continue
#     if pd.notna(systems.loc[idx, 'Description']) and (systems.loc[idx+1, 'Format'] == 'Lineitem')\
#         and pd.notna(systems.loc[idx+1, 'SN']) and pd.isnull(systems.loc[idx-1, 'Description'])\
#         and pd.isnull(systems.loc[idx, 'SN']) and (systems.loc[idx, 'NO'] != 'Title'):
#         systems.at[idx, 'Format'] = 'Subtitle'
#     if idx == len(systems)-2:
#         break

In [8]:
# Move Option and Included to scope
for idx in systems.index:
    if str(systems.loc[idx, 'Subtotal Price']).lower() in ['option', 'optional']:
        systems.at[idx, 'Scope'] = 'OPTION'
    if str(systems.loc[idx, 'Subtotal Price']).lower() in ['included','inclusive']:
        systems.at[idx, 'Scope'] = 'INCLUDED'
        

In [9]:
# Cleaning data
for idx in systems.index:
    if functions.set_nitty_gritty(str(systems.loc[idx, 'Description'])) != 'None':
        systems.at[idx, 'Description'] = functions.set_nitty_gritty(str(systems.loc[idx, 'Description']))
    if (str(systems.loc[idx, 'Model']).lower().strip() == 'start line:  delete forbidden'):
        systems.at[idx, 'Model'] = np.nan
    if (str(systems.loc[idx, 'UC']).lower().strip() == 'true' or str(systems.loc[idx, 'UC']).lower().strip() == 'false'):
        systems.at[idx, 'UC'] = np.nan   
    if (str(systems.loc[idx, 'SC']).lower().strip() == 'true' or str(systems.loc[idx, 'SC']).lower().strip() == 'false'):
        systems.at[idx, 'SC'] = np.nan
    if (str(systems.loc[idx, 'Model']).lower().strip() == 'true' or str(systems.loc[idx, 'Model']).lower().strip() == 'false'):
        systems.at[idx, 'Model'] = np.nan   

In [10]:
# Download template file from the internet and write to local folder
url = "https://filedn.com/liTeg81ShEXugARC7cg981h/Proposal_Template.xlsx"
resp = requests.get(url)

with open("Template.xlsx", 'wb') as fd:
    for chunk in resp.iter_content(chunk_size=8192):
        fd.write(chunk)

In [11]:
# Copy sheet from template to new workbook
nb = xw.Book()
template = xw.Book("Template.xlsx", password=hide.new)
template.sheets['config'].copy(after=nb.sheets[0])
nb.sheets['Sheet1'].delete()
template.sheets['Cover'].copy(after=nb.sheets['config'])

# Set date in Config
nb.sheets['Config'].range('B32').value = datetime.today().strftime('%Y-%m-%d')

# Set up formula in Cover sheet
nb.sheets['Cover'].range('D7').formula = '=Config!B26'
nb.sheets['Cover'].range('C42').formula = '=Config!B21'
nb.sheets['Cover'].range('C43').formula = '=Config!B23'
nb.sheets['Cover'].range('C44').formula = '=Config!B24'
nb.sheets['Cover'].range('C45').formula = '=Config!B29'
nb.sheets['Cover'].range('C46').formula = '=Config!B30'
nb.sheets['Cover'].range('C47').formula = '=Config!B32'
nb.sheets['Cover'].range('D39').formula = '=Config!B13'

for system in system_names[::-1]:
    sheet_name = 'Cover'
    template.sheets['System'].copy(after=nb.sheets[sheet_name])
    sheet_name = system
    nb.sheets['System'].name = sheet_name
    # Set formula to reference Config.
    # nb.sheets[sheet_name].range('C1').formula = '=Config!B29'
    # nb.sheets[sheet_name].range('C2').formula = '=Config!B30'
    # nb.sheets[sheet_name].range('C3').formula = '=Config!B32'
    # nb.sheets[sheet_name].range('C4').formula = '=Config!B26'
    nb.sheets[sheet].range('A1').formula = '= "JASON REF: " & Config!B29 &  ", REVISION: " &  Config!B30 & ", PROJECT: " & Config!B26'
template.sheets['Summary'].copy(after=nb.sheets['Cover'])
template.sheets['Technical_Notes'].copy(after=nb.sheets[-1])
template.sheets['T&C'].copy(after=nb.sheets[-1])
for sheet in nb.sheet_names:
    if sheet in ['Summary','Technical_Notes', 'T&C']:
        # nb.sheets[sheet].range('C1').formula = '=Config!B29'
        # nb.sheets[sheet].range('C2').formula = '=Config!B30'
        # nb.sheets[sheet].range('C3').formula = '=Config!B32'
        # nb.sheets[sheet].range('C4').formula = '=Config!B26'
        nb.sheets[sheet].range('A1').formula = '= "JASON REF: " & Config!B29 &  ", REVISION: " &  Config!B30 & ", PROJECT: " & Config!B26'
template.close()

In [12]:
for system in system_names:
    sheet = nb.sheets[system]
    system = systems[systems['System'] == system]
    sheet.range('A2').options(index=False).value = system

In [13]:
# Set exchange rates
sheet = nb.sheets['Config']
exchange = pd.DataFrame([exchange_rates])
exchange = exchange.T
sheet.range('A2').value = exchange

# Quoted currency
sheet.range('B12').value = quoted_currency

# Project info
sheet.range('B21').value = project_info['Attend to: ']
sheet.range('B22').value = project_info['Designation: ']
sheet.range('B23').value = project_info['Client Name: ']
sheet.range('B24').value = project_info['Client RFQ No: ']
sheet.range('B25').value = project_info['Ref Doc No: ']
sheet.range('B26').value = project_info['Project Name: ']
sheet.range('B27').value = project_info['Prepared By: ']
sheet.range('B28').value = project_info['Sales Manager: ']
sheet.range('B29').value = project_info['Jason Ref: ']
sheet.range('B30').value = project_info['Revision Num: ']
# sheet.range('B31').value = project_info['']
sheet.range('B32').value = datetime.today().strftime('%Y-%m-%d')

In [14]:
# Write necessary formula to excel
for system in system_names:
    sheet = nb.sheets[system]

    # Set default values
    if system != 'ENGINEERING SERVICES':
        sheet.range('J1').value = defaults[system]['default_mu']
        sheet.range('L1').value = defaults[system]['Default']
        sheet.range('N1').value = defaults[system]['Warranty']
        sheet.range('P1').value = defaults[system]['Inbound Freight']
        sheet.range('R1').value = defaults[system]['Special Terms']
    else:
        sheet.range('J1').value = 0.3
        sheet.range('L1').value = 0
        sheet.range('N1').value = 0
        sheet.range('P1').value = 0
        sheet.range('R1').value = 0
    
    functions.fill_formula(sheet)


In [15]:
# Summary
functions.fill_lastrow(nb)
functions.summary(nb)

In [16]:
# Setup print area
for system in system_names:
    sheet = nb.sheets[system]
    functions.format(sheet)
    last_row = sheet.range('G100000').end('up').row
    sheet.page_setup.print_area = 'A1:H' + str(last_row)

In [17]:
nb.sheets['Summary'].activate()
nb.save( downloads_folder + '/' + file[:-4] + 'xlsx', password=hide.legacy) 
# nb.close()